In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score , confusion_matrix

import itertools
from google.colab import drive
import os

# This will prompt for authorization.
drive.mount('/content/drive')

# Specify the path to your folder
folder_path = '/content/drive/My Drive/Final Training Data'
dataPath= os.path.join(os.getcwd(), 'drive', 'My Drive', 'Final Training Data')
# Define the base directory where your class folders are located
base_dir = dataPath

# Define the destination base directories
train_dir = os.path.join(dataPath, "Train")
test_dir = os.path.join(dataPath, "Test")
valid_dir = os.path.join(dataPath, "Valid")

BATCH_SIZE = 32
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
train_dataset = keras.utils.image_dataset_from_directory(
    os.path.join(dataPath, 'Train'),
    image_size = (IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size = BATCH_SIZE,
    label_mode = 'categorical'
)

valid_dataset = keras.utils.image_dataset_from_directory(
    os.path.join(dataPath, 'Valid'),
    image_size = (IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size = BATCH_SIZE,
    label_mode = 'categorical'
)

test_dataset = keras.utils.image_dataset_from_directory(
    os.path.join(dataPath, 'Test'),
    image_size = (IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size = BATCH_SIZE,
    label_mode = 'categorical',
    shuffle=False
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 9600 files belonging to 4 classes.
Found 1200 files belonging to 4 classes.
Found 1200 files belonging to 4 classes.


In [2]:
from tensorflow.keras import models
from keras.layers import Dense, Dropout, Flatten
data_augmentation = keras.Sequential([
  layers.RandomFlip("horizontal"), # Applies horizontal flipping to a random 50% of the images
  layers.RandomRotation(0.1), # Rotates the input images by a random value in the range[–10%, +10%] (fraction of full circle [-36°, 36°])
  layers.RandomZoom(0.1), # Zooms in or out of the image by a random factor in the range [-20%, +20%]
  layers.RandomContrast(0.1),
],
name = "AugmentationLayer"
)

model = keras.Sequential(name="FromScratch")
model.add(data_augmentation)
model.add(layers.Rescaling(1./255))

model.add(layers.Conv2D(32, kernel_size = 3, activation = "relu6", padding = "same", input_shape = (256, 256,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(64, kernel_size = 3, activation='relu', padding = "same"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(128, kernel_size = 3, activation='relu', padding = "same"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(256, kernel_size = 3, activation='relu', padding = "same"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(512, kernel_size = 3, activation='relu', padding = "same"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(512, kernel_size = 3, activation='relu', padding = "same"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(256,activation="relu"))
model.add(layers.Dense(4,activation="softmax"))

model.build(input_shape=(None, 256, 256, 3))
model.add(Flatten())
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

model.summary()

Model: "FromScratch"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 AugmentationLayer (Sequent  (None, 256, 256, 3)       0         
 ial)                                                            
                                                                 
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 32)      0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 128, 128, 32)      128       
 Normalization)                                                  
                                                       

In [4]:
import tensorflow as tf
from google.colab import files
# Convert the model.
modelName = "fin"
scratchPath=scratchPath = os.path.join(dataPath, 'models', 'from_scratch')
modelPath = os.path.join(scratchPath, modelName + ".keras")
model=tf.keras.models.load_model(modelPath, compile=False)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)
files.download('model.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>